# Finding the tasks and activities that raise and lower automation risk

To develop a better understanding of what drives automation risk, we analysed the types of tasks and work activities that lowered or raised the risk of automation. The impact of each task was estimated by calculating
how the automation scores of occupations would change if the task were removed. To aid interpretation of results, individual tasks were grouped into detailed work activities (DWAs), intermediate work activities (IWAs) and Elements using the O\*NET framework.

# 0. Set up dependencies and helper functions

In [1]:
%run ../notebook_preamble.ipy

import os
import collections

In [2]:
# Folder that stores interim outputs
interim_outputs = f'{data_folder}interim/automation_analysis/'

In [3]:
# Columns that refer to SML dimensions assessed by Brynjolfsson et al. rubric
data_cols = [
    'q1', 'q2', 'q3', 'q4', 'q5',
    'q6', 'q7', 'q8', 'q9', 'q10',
    'q11', 'q12', 'q13', 'q14', 'qD',
    'q19', 'q20', 'q21', 'q22', 'q23']

In [4]:
# Calculate mean SML score across all questions for a given task
def get_task_meanSML(df_row):
    mSML = np.mean(df_row[data_cols])
    return(mSML)

# Identify number of SML dimensions that have low SML scores
def is_bottleneck(df_row):
    bottlenecks = [elem for elem in df_row[data_cols] if elem <=2]
    return (len(bottlenecks))

#Check whether there are any bottlenecks in a given task
def has_bottlenecks(col_value):
    if col_value >0:
        res = True
    else:
        res = False
    return res

# 1. Calculate baseline task SML scores

In [5]:
# Import task SML scores crosswalked to ESCO occupations
job_task_scores_esco = pd.read_csv(os.path.join(interim_outputs + 'job_task_scores_esco.csv'))

# Organise the dataframe
job_task_scores_esco = job_task_scores_esco[[
    'O*NET-SOC Code', 'Title', 'Task ID', 'Task', 'Data Value',
    'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 
    'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20',
    'q21', 'qD', 'q22', 'q23',
    'weight','id', 'esco_occupation', 'isco_code']]

In [6]:
# Calculate mean task risk and multipy by its weight
job_task_scores_esco['mean_risk'] = job_task_scores_esco.apply(get_task_meanSML, axis=1)


In [7]:
job_task_scores_esco.head(3)

,O*NET-SOC Code,Title,Task ID,Task,Data Value,q1,q2,q3,q4,q5,...,q20,q21,qD,q22,q23,weight,id,esco_occupation,isco_code,mean_risk
0,11-1011.00,chief executives,8823,Direct or coordinate an organization's financi...,0.908,3.0,3.0,2.5,3.0,3.0,...,4.0,3.5,4.0,3.0,3.0,0.046794,188.0,secretary of state,1111.0,3.325
1,11-1011.00,chief executives,8823,Direct or coordinate an organization's financi...,0.908,3.0,3.0,2.5,3.0,3.0,...,4.0,3.5,4.0,3.0,3.0,0.046794,775.0,government minister,1111.0,3.325
2,11-1011.00,chief executives,8823,Direct or coordinate an organization's financi...,0.908,3.0,3.0,2.5,3.0,3.0,...,4.0,3.5,4.0,3.0,3.0,0.046794,1104.0,member of parliament,1111.0,3.325


# 2. Evaluate impact of tasks and work activities on occupational risk

We performed the following steps: 

(1) Mapped tasks to DWAs then to IWAs and to Elements (general work activities; GWAs).  

(2) Iteratively dropped tasks from the full dataframe, then recalculated occupation level risk (measured as weighted average task risk). The task *impact* was calculated as the difference between the *recalculated* occupation level risk and the *baseline* occupation level risk.  

**Note:** tasks that increase the risk will have a positive-valued impact whereas tasks that decrease risk will have a negative-valued impact.  

(3) Aggregated the results in the following way:  
- Summed impacts for the same task across occupations<br>
- Summed impacts of all tasks that live in the same DWAs<br>
- For subsequent analysis of IWAs and Elements, we used median DWA scores

In [8]:
# Import occupational risk scores
agg_job_task_scores_esco = pd.read_csv(interim_outputs + 'agg_job_task_scores_esco_revised_w_risk_q.csv')

# Import mapping between tasks, IWAs and DWAs
# (using the exact same files that were used in the original analysis, retrieved in Spring 2020 from O*NET)
task_dwa_mapping = pd.read_csv(f'{interim_outputs}/lookups/Tasks to DWAs.csv')
dwa_iwa_mapping = pd.read_csv(f'{interim_outputs}/lookups/dwa_to_iwa_to_elements.csv')

## 2.1 Map tasks to DWAs, IWAs and Elements

In [9]:
# Helper dictionaries
task_to_dwa = {row['Task']: row['DWA Title'] for ix, row in task_dwa_mapping.iterrows()}
dwa_to_iwa = {row['DWA Title']: row['IWA Title'] for ix, row in dwa_iwa_mapping.iterrows()}
iwa_to_gwa = {row['IWA Title']: row['Element Name'] for ix, row in dwa_iwa_mapping.iterrows()}

# Edge cases
dwa_to_iwa['Clean work areas or facilities.'] = 'na'
dwa_to_iwa['Collect payments for good or services.'] = 'na'
dwa_to_iwa['Research social issues.'] = 'na'

iwa_to_gwa['na'] = 'na'

## 2.2 Calculate task impacts

In [10]:
# Calculate baseline occupational risk
baseline_risk_lookup = dict()
for ix, row in agg_job_task_scores_esco.iterrows():
    occupation = row['esco_occupation']
    baseline_risk = row['weighted_risk']
    baseline_risk_lookup[occupation] = baseline_risk

In [11]:
# List of unique tasks
task_list = job_task_scores_esco['Task'].unique() 
indexed_job_task_scores_t = job_task_scores_esco.set_index('Task')

In [12]:
# This dictionary records occupations where a given task is required. 
# Impact of tasks is then only checked for these occupations.
task_occupations = collections.defaultdict(list)
for ix, row in indexed_job_task_scores_t.iterrows():
    occup = row['esco_occupation']
    task_occupations[ix].append(occup)

In [13]:
# Impact of all tasks;
# WARNING: takes a few hours to run through; alternatively, jump to the next cell and load precomputed impacts
task_risk_diffs = collections.defaultdict(dict)
for ix, task in enumerate(task_list):
    print(ix, task)
    occupations_to_check = task_occupations[task]
    updated_task_risk = indexed_job_task_scores_t.drop(task)
    occupation_risk_task = dict()

    for name, group in updated_task_risk.groupby('esco_occupation'):
        if name in occupations_to_check:
            new_total_weight = group['weight'].sum()
            group['new_weight'] = group['weight']/new_total_weight
            group['new_weighted_risk'] = group['mean_risk']*group['new_weight']
            new_occup_risk = group['new_weighted_risk'].sum()
            baseline_risk = baseline_risk_lookup[name]
            diff = new_occup_risk - baseline_risk
            occupation_risk_task[name] = diff
        else:
            occupation_risk_task[name] = 0.0
    
    task_risk_diffs[task] = occupation_risk_task

0 Direct or coordinate an organization's financial or budget activities to fund operations, maximize investments, or increase efficiency.


/Users/karliskanders/opt/miniconda3/envs/mapping_career_causeways/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/karliskanders/opt/miniconda3/envs/mapping_career_causeways/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



1 Appoint department heads or managers and assign or delegate responsibilities to them.
2 Analyze operations to evaluate performance of a company or its staff in meeting objectives or to determine areas of potential cost reduction, program improvement, or policy change.
3 Direct, plan, or implement policies, objectives, or activities of organizations or businesses to ensure continuing operations, to maximize returns on investments, or to increase productivity.
4 Prepare budgets for approval, including those for funding or implementation of programs.
5 Confer with board members, organization officials, or staff members to discuss issues, coordinate activities, or resolve problems.
6 Implement corrective action plans to solve organizational or departmental problems.
7 Direct human resources activities, including the approval of human resource plans or activities, the selection of directors or other high-level staff, or establishment or organization of major departments.
8 Negotiate or ap

74 Manage sales team including setting goals, providing incentives, and evaluating employee performance.
75 Prepare and negotiate advertising and sales contracts.
76 Formulate plans to extend business with established accounts and to transact business as agent for advertising accounts.
77 Train and direct workers engaged in developing and producing advertisements.
78 Confer with clients to provide marketing or technical advice.
79 Assemble and communicate with a strong, diverse coalition of organizations or public figures, securing their cooperation, support, and action, to further campaign goals.
80 Provide presentation and product demonstration support during the introduction of new products and services to field staff and customers.
81 Represent company at trade association meetings to promote products.
82 Direct and coordinate product research and development.
83 Identify, develop, or evaluate marketing strategy, based on knowledge of establishment objectives, market characteristic

150 Direct daily operations of department, analyzing workflow, establishing priorities, developing standards and setting deadlines.
151 Meet with department heads, managers, supervisors, vendors, and others, to solicit cooperation and resolve problems.
152 Review project plans to plan and coordinate project activity.
153 Assign and review the work of systems analysts, programmers, and other computer-related workers.
154 Provide users with technical support for computer problems.
155 Develop computer information resources, providing for data security and control, strategic computing, and disaster recovery.
156 Recruit, hire, train and supervise staff, or participate in staffing decisions.
157 Stay abreast of advances in technology.
158 Consult with users, management, vendors, and technicians to assess computing needs and system requirements.
159 Develop and interpret organizational goals, policies, and procedures.
160 Evaluate the organization's technology use and needs and recommend im

...

16805 Oversee operation of vessels used for carrying passengers, motor vehicles, or goods across rivers, harbors, lakes, and coastal waters.
16806 Secure boats to docks with mooring lines, and cast off lines to enable departure.
16807 Service motors by performing tasks such as changing oil and lubricating parts.
16808 Arrange repairs, fuel, and supplies for vessels.
16809 Maintain equipment such as range markers, fire extinguishers, boat fenders, lines, pumps, and fittings.
16810 Organize and direct the activities of crew members.
16811 Report any observed navigational hazards to authorities.
16812 Issue directions for loading, unloading, and seating in boats.
16813 Clean boats and repair hulls and superstructures, using hand tools, paint, and brushes.
16814 Tow, push, or guide other boats, barges, logs, or rafts.
16815 Take depth soundings in turning basins.
16816 Position booms around docked ships.
16817 Start engines to propel ships and regulate engines and power transmissions to co

16888 Measure and record the speed of vehicular traffic, using electrical timing devices or radar equipment.
16889 Operate counters and record data to assess the volume, type, and movement of vehicular or pedestrian traffic at specified times.
16890 Review traffic control or barricade plans to issue permits for parades or other special events or for construction work that affects rights of way, providing assistance with plan preparation or revision, as necessary.
16891 Time stoplights or other delays, using stopwatches.
16892 Visit development or work sites to determine projects' effect on traffic and the adequacy of traffic control and safety plans or to suggest traffic control measures.
16893 Establish procedures for street closures or for repair or construction projects.
16894 Provide traffic information, such as road conditions, to the public.
16895 Monitor street or utility projects for compliance to traffic control permit conditions.
16896 Develop plans or long-range strategies f

16963 Transport baggage or coordinate transportation between assigned rooms, terminals, or platforms.
16964 Inform supervisors of equipment malfunctions that need to be addressed.
16965 Observe conveyor operations and monitor lights, dials, and gauges to maintain specified operating levels and to detect equipment malfunctions.
16966 Record production data such as weights, types, quantities, and storage locations of materials, as well as equipment performance problems and downtime.
16967 Load, unload, or adjust materials or products on conveyors by hand, by using lifts, hoists, and scoops, or by opening gates, chutes, or hoppers.
16968 Stop equipment or machinery and clear jams, using poles, bars, and hand tools, or remove damaged materials from conveyors.
16969 Distribute materials, supplies, and equipment to work stations, using lifts and trucks.
16970 Collect samples of materials or products, checking them to ensure conformance to specifications or sending them to laboratories for an

17038 Apply hand or foot brakes and move levers to lock hoists or winches.
17039 Oil winch drums so that cables will wind smoothly.
17040 Climb ladders to position and set up vehicle-mounted derricks.
17041 Repair, maintain, and adjust equipment, using hand tools.
17042 Tend auxiliary equipment, such as jacks, slings, cables, or stop blocks, to facilitate moving items or materials for further processing.
17043 Move levers or controls that operate lifting devices, such as forklifts, lift beams with swivel-hooks, hoists, or elevating platforms, to load, unload, transport, or stack material.
17044 Inspect product load for accuracy and safely move it around the warehouse or facility to ensure timely and complete delivery.
17045 Manually or mechanically load or unload materials from pallets, skids, platforms, cars, lifting devices, or other transport vehicles.
17046 Position lifting devices under, over, or around loaded pallets, skids, or boxes and secure material or products for transport 

17119 Submit daily reports on facility operations.
17120 Take samples of gases and conduct chemical tests to determine gas quality and sulfur or moisture content, or send samples to laboratories for analysis.
17121 Read gas meters, and maintain records of the amounts of gas received and dispensed from holders.
17122 Turn knobs or switches to regulate pressures.
17123 Clean, lubricate, and adjust equipment, and replace filters and gaskets, using hand tools.
17124 Maintain each station by performing general housekeeping duties such as painting, washing, and cleaning.
17125 Connect pipelines between pumps and containers that are being filled or emptied.
17126 Monitor gauges and flowmeters and inspect equipment to ensure that tank levels, temperatures, chemical amounts, and pressures are at specified levels, reporting abnormalities as necessary.
17127 Record operating data such as products and quantities pumped, stocks used, gauging results, and operating times.
17128 Communicate with othe

In [28]:
file_path = interim_outputs + 'task_risk_diffs.pkl'
if os.path.exists(file_path) == False:
    with open(file_path, 'wb') as f:
        pickle.dump(task_risk_diffs, f)
else:
    with open(file_path, 'rb') as f:
        task_risk_diffs = pickle.load(f)

In [29]:
# Number of occupations in which a given task is required
importances = []
for elem in list(xx.keys()):
    importances.append((elem, len(task_occupations[elem])))

In [30]:
# Aggregate contributions of a given task across all occupations in which it is required
task_impact = {k: sum(v.values()) for k,v in xx.items()}

In [31]:
len(task_impact)

17193

In [32]:
# Convert impacts to a dataframe and show corresponding DWAs, IWAs and Elements
task_impact_combined = pd.DataFrame({'task': list(task_impact.keys()), 
                                     'impact': list(task_impact.values())})

task_impact_combined['importance'] = task_impact_combined['task'].apply(lambda x: len(task_occupations[x]))

task_impact_combined['dwa'] = task_impact_combined['task'].apply(lambda x: task_to_dwa[x])

task_impact_combined['iwa'] = task_impact_combined['dwa'].apply(lambda x: dwa_to_iwa[x])

task_impact_combined['gwa'] = task_impact_combined['iwa'].apply(lambda x: iwa_to_gwa[x])

In [33]:
task_impact_combined

,task,impact,importance,dwa,iwa,gwa
0,Direct or coordinate an organization's financi...,0.120172,14,Direct financial operations.,Manage budgets or finances.,"Guiding, Directing, and Motivating Subordinates"
1,Appoint department heads or managers and assig...,0.016842,14,Prepare staff schedules or work assignments.,Assign work to others.,"Guiding, Directing, and Motivating Subordinates"
2,Analyze operations to evaluate performance of ...,-0.083235,14,Analyze data to assess operational or project ...,Analyze data to improve operations.,Analyzing Data or Information
3,"Direct, plan, or implement policies, objective...",-0.204679,14,"Direct organizational operations, projects, or...","Direct organizational operations, activities, ...","Guiding, Directing, and Motivating Subordinates"
4,"Prepare budgets for approval, including those ...",-0.204440,14,Prepare operational budgets.,Manage budgets or finances.,"Guiding, Directing, and Motivating Subordinates"
...,...,...,...,...,...,...
17188,"Test vessels for leaks, damage, and defects, a...",0.000000,1,Maintain material moving equipment in good wor...,Maintain vehicles in working condition.,Repairing and Maintaining Mechanical Equipment
17189,Unload cars containing liquids by connecting h...,0.000000,1,Connect hoses to equipment or machinery.,Connect components or supply lines to equipmen...,Handling and Moving Objects
17190,Copy and attach load specifications to loaded ...,0.000000,1,Mark materials or objects for identification.,Mark materials or objects for identification.,"Identifying Objects, Actions, and Events"
17191,Start pumps and adjust valves or cables to reg...,0.000000,1,Control pumps or pumping equipment.,Operate pumping systems or equipment.,Controlling Machines and Processes


## 2.3 Aggregate task impacts

In [34]:
# Aggregate impact of tasks that reside within DWAs
dwa_impact = task_impact_combined.groupby('dwa').agg({'impact': sum, 
                                                      'importance': lambda x: len(x)})
dwa_impact['iwa'] = dwa_impact.index.map(lambda x: dwa_to_iwa[x])
dwa_impact['gwa'] = dwa_impact['iwa'].apply(lambda x: iwa_to_gwa[x])

In [35]:
dwa_impact.head()

,impact,importance,iwa,gwa
dwa,,,,
Accompany individuals or groups to activities.,-0.027533,4,Escort others.,Performing General Physical Activities
Accompany patients or clients on outings to provide assistance.,0.034211,4,Escort others.,Performing General Physical Activities
Acquire supplies or equipment.,-0.284714,7,Purchase goods or services.,Monitoring and Controlling Resources
Add garnishes to food.,-0.005975,2,Prepare foods or beverages.,Handling and Moving Objects
Adjust dental devices or appliances to ensure fit.,-0.049506,5,Adjust medical equipment to ensure adequate pe...,Handling and Moving Objects


In [36]:
# Export raw results to interim outputs 
dwa_impact.to_csv(interim_outputs + 'dwa_impact.csv')


### Check examples

These correspond to Table 4 (page 46) in the Mapping Career Causeways report.

In [37]:
# Most risky general activities (Elements)
dwa_impact.groupby('gwa').agg({'impact': 'median'}).sort_values('impact').head(5)

,impact
gwa,
Interacting With Computers,-0.036205
Communicating with Persons Outside Organization,-0.032265
Selling or Influencing Others,-0.026149
Monitoring and Controlling Resources,-0.025074
Analyzing Data or Information,-0.023200


In [38]:
# Least risky general activities (Elements)
dwa_impact.groupby('gwa').agg({'impact': 'median'}).sort_values('impact', ascending=False).head(5)

,impact
gwa,
Performing for or Working Directly with the Public,0.094476
"Operating Vehicles, Mechanized Devices, or Equipment",0.093936
Updating and Using Relevant Knowledge,0.079790
Resolving Conflicts and Negotiating with Others,0.034814
Controlling Machines and Processes,0.027305


# 3. Prepare and export the final tables on task and DWA impacts

## 3.1 Export task impacts

In [39]:
# Get raw task info, specifically mean SML risk and number of bottlenecks
raw_task_info = job_task_scores_esco.drop_duplicates(subset = 'Task')
raw_task_info = raw_task_info[[
    'Task ID', 'Task', 'Data Value', 'q1', 'q2',
    'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13',
    'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20', 'q21', 'qD', 'q22', 'q23',
    'weight', 'mean_risk']]

raw_task_info['num_bottlenecks'] = raw_task_info.apply(is_bottleneck, axis =1)
raw_task_info['has_bottlenecks'] = raw_task_info['num_bottlenecks'].apply(lambda x: has_bottlenecks(x))

In [40]:
# Merge raw task info with task impact
combined_task_info = task_impact_combined.merge(
    raw_task_info, how = 'inner',
    right_on = 'Task', left_on = 'task')

In [41]:
combined_task_info = combined_task_info[[
    'Task ID', 'Task','dwa', 'iwa', 'gwa',
    'impact', 'importance',
    'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
    'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20',
    'q21', 'qD', 'q22', 'q23', 
    'weight', 'mean_risk',
    'num_bottlenecks', 'has_bottlenecks']]

In [43]:
# Export raw results to interim outputs
combined_task_info.to_csv(interim_outputs + 'combined_task_info.csv', index = False)
combined_task_info.head(3)

,Task ID,Task,dwa,iwa,gwa,impact,importance,q1,q2,q3,...,q19,q20,q21,qD,q22,q23,weight,mean_risk,num_bottlenecks,has_bottlenecks
0,8823,Direct or coordinate an organization's financi...,Direct financial operations.,Manage budgets or finances.,"Guiding, Directing, and Motivating Subordinates",0.120172,14,3.0,3.0,2.5,...,3.0,4.0,3.5,4.0,3.0,3.0,0.046794,3.325,0,False
1,8831,Appoint department heads or managers and assig...,Prepare staff schedules or work assignments.,Assign work to others.,"Guiding, Directing, and Motivating Subordinates",0.016842,14,3.5,3.5,4.0,...,3.5,3.5,3.5,4.0,3.5,3.0,0.046176,3.475,0,False
2,8825,Analyze operations to evaluate performance of ...,Analyze data to assess operational or project ...,Analyze data to improve operations.,Analyzing Data or Information,-0.083235,14,4.0,3.0,3.0,...,3.5,4.0,4.0,4.0,2.0,2.0,0.045351,3.625,2,True


### 3.1.1 Table to be published as the Supplementary online data

In [44]:
# Select and order the columns
Table_Task_impact = combined_task_info.copy()

Table_Task_impact = Table_Task_impact[[
    'Task ID',
    'Task',
    'dwa',
    'iwa',
    'gwa',
    'mean_risk',
    'impact',
    'q1', 'q2', 'q3', 'q4', 'q5',
    'q6', 'q7', 'q8', 'q9', 'q10',
    'q11', 'q12', 'q13', 'q14',
    'q15', 'q16', 'q17', 'q18', 
    'qD',
    'q19', 'q20', 'q21', 'q22', 'q23'
]]

# Rename the columns
Table_Task_impact = Table_Task_impact.rename(columns = {
    'Task ID': 'task_id',
    'Task': 'task',
    'dwa': 'detailed_work_activity',
    'iwa': 'intermediate_work_activity',
    'gwa': 'element',
    'mean_risk': 'mean_task_SML',
})

Table_Task_impact = Table_Task_impact.sort_values('task_id')

In [45]:
# Export the final table
Table_Task_impact.to_csv(useful_paths.project_dir + '/supplementary_online_data/automation_risk/mcc_risk_Tasks.csv',
                         index=False)


## 3.2 Export results aggregated at the DWA level

In [46]:
# Select and organise the columns
Table_DWA_impact = dwa_impact.reset_index()[[
    'dwa', 'impact', 'iwa', 'gwa'
]].copy()

# Rename the columns
Table_DWA_impact = Table_DWA_impact.rename(columns = {
    'dwa': 'detailed_work_activity',
    'iwa': 'intermediate_work_activity',
    'gwa': 'element',
})

Table_DWA_impact

,detailed_work_activity,impact,intermediate_work_activity,element
0,Accompany individuals or groups to activities.,-0.027533,Escort others.,Performing General Physical Activities
1,Accompany patients or clients on outings to pr...,0.034211,Escort others.,Performing General Physical Activities
2,Acquire supplies or equipment.,-0.284714,Purchase goods or services.,Monitoring and Controlling Resources
3,Add garnishes to food.,-0.005975,Prepare foods or beverages.,Handling and Moving Objects
4,Adjust dental devices or appliances to ensure ...,-0.049506,Adjust medical equipment to ensure adequate pe...,Handling and Moving Objects
...,...,...,...,...
2057,Write grant proposals.,-0.298284,Prepare proposals or grant applications.,Documenting/Recording Information
2058,Write informational material.,-0.296591,Prepare informational or instructional materials.,Documenting/Recording Information
2059,Write material for artistic or entertainment p...,0.219941,Write material for artistic or commercial purp...,Documenting/Recording Information
2060,Write operational reports.,-0.083607,Prepare reports of operational or procedural a...,Documenting/Recording Information


In [47]:
# Export the table for the Supplementary online data
Table_DWA_impact.to_csv(useful_paths.project_dir + '/supplementary_online_data/automation_risk/mcc_risk_DWAs.csv',
                        index=False)


# 4. Estimate contribution of SML dimensions to bottleneck tasks

In [48]:
with_bottlenecks = raw_task_info[raw_task_info['has_bottlenecks'] == True]
qs = with_bottlenecks[data_cols].T
qs['is_bottleneck'] = qs.apply(lambda x: len([elem for elem in x if elem <=2]), axis =1)
short_qs = qs['is_bottleneck']

In [49]:
short_qs

q1      291
q2      242
q3      930
q4      200
q5     1334
q6     1112
q7      916
q8      323
q9     1050
q10     343
q11     255
q12     612
q13     864
q14     766
qD       51
q19     943
q20     217
q21     216
q22    5859
q23    6457
Name: is_bottleneck, dtype: int64

In [50]:
short_qs.to_csv(interim_outputs + 'sml_dimension_contrib_bottlenecks.csv', index=False)